<a href="https://colab.research.google.com/github/someshsingh22/TeamEleven/blob/main/ClassificationReportAndNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report

In [3]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 5.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [ ]:
np.random.seed(60)

#Classification reports using the Decision Tree Regressor

In [69]:
# Access data store
data_store = pd.HDFStore('flipped_data.h5')

# Retrieve data using key
flipped_df = data_store['flip_df']
data_store.close()

In [70]:
flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG,1,0.64,-0.64
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN,2,-0.14,0.14


In [71]:
len(flipped_df)

7298

In [72]:
enc = OneHotEncoder(handle_unknown='ignore')

In [73]:
enc_df = pd.DataFrame(enc.fit_transform(flipped_df[['Winner_team','TOSS']]).toarray())

In [74]:
flipped_df = flipped_df.join(enc_df)
flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2,0,1,2,3
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5,0.0,1.0,1.0,0.0
0,2015-03-04,AUS,AFG,2,3773,65,2,417,142,AUS,1,5.5,-5.5,0.0,1.0,1.0,0.0
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32,0.0,1.0,0.0,1.0
1,2012-08-25,AUS,AFG,1,3437,156,1,272,206,AUS,1,1.32,-1.32,0.0,1.0,0.0,1.0
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04,0.0,1.0,1.0,0.0


In [75]:
flipped_df.rename(columns = {0: 'Win1',1: 'Win2',2: 'Toss1',3: 'Toss2'}, inplace = True)

In [76]:
labelencoder = LabelEncoder()

flipped_df['Team_1Enc'] = labelencoder.fit_transform(flipped_df['Team_1'])
flipped_df['Team_2Enc'] = labelencoder.fit_transform(flipped_df['Team_2'])

In [77]:
flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2,Win1,Win2,Toss1,Toss2,Team_1Enc,Team_2Enc
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5,0.0,1.0,1.0,0.0,0,1
0,2015-03-04,AUS,AFG,2,3773,65,2,417,142,AUS,1,5.5,-5.5,0.0,1.0,1.0,0.0,1,0
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32,0.0,1.0,0.0,1.0,0,1
1,2012-08-25,AUS,AFG,1,3437,156,1,272,206,AUS,1,1.32,-1.32,0.0,1.0,0.0,1.0,1,0
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04,0.0,1.0,1.0,0.0,0,1


In [78]:
team_group = flipped_df.groupby(["Team_1", "Team_2"]).size().reset_index(name="Matches")

In [79]:
team_group.head()

,Team_1,Team_2,Matches
0,AFG,AUS,3
1,AFG,BAN,8
2,AFG,ENG,1
3,AFG,IND,3
4,AFG,IRE,26


In [80]:
teams_list = team_group.values.tolist()

In [81]:
print(teams_list)

[['AFG', 'AUS', 3], ['AFG', 'BAN', 8], ['AFG', 'ENG', 1], ['AFG', 'IND', 3], ['AFG', 'IRE', 26], ['AFG', 'KEN', 6], ['AFG', 'NED', 6], ['AFG', 'NZL', 2], ['AFG', 'PAK', 4], ['AFG', 'SCO', 10], ['AFG', 'SRL', 3], ['AFG', 'WIN', 8], ['AFG', 'ZIM', 22], ['AUS', 'AFG', 3], ['AUS', 'BAN', 21], ['AUS', 'ENG', 145], ['AUS', 'IND', 128], ['AUS', 'IRE', 3], ['AUS', 'KEN', 5], ['AUS', 'NED', 1], ['AUS', 'NZL', 129], ['AUS', 'PAK', 99], ['AUS', 'SAF', 98], ['AUS', 'SCO', 5], ['AUS', 'SRL', 85], ['AUS', 'WIN', 128], ['AUS', 'ZIM', 29], ['BAN', 'AFG', 8], ['BAN', 'AUS', 21], ['BAN', 'ENG', 21], ['BAN', 'IND', 32], ['BAN', 'IRE', 9], ['BAN', 'KEN', 13], ['BAN', 'NED', 2], ['BAN', 'NZL', 32], ['BAN', 'PAK', 33], ['BAN', 'SAF', 20], ['BAN', 'SCO', 4], ['BAN', 'SRL', 41], ['BAN', 'WIN', 37], ['BAN', 'ZIM', 73], ['ENG', 'AFG', 1], ['ENG', 'AUS', 145], ['ENG', 'BAN', 21], ['ENG', 'IND', 91], ['ENG', 'IRE', 10], ['ENG', 'KEN', 2], ['ENG', 'NED', 3], ['ENG', 'NZL', 82], ['ENG', 'PAK', 86], ['ENG', 'SAF', 5

##Classification Report on Team vs Team

In [96]:
def print_model_scores(model, data, y, predictors, target):
    '''
    A generic function to generate the performance report of the
    model in question on the data passed to it
    Args:
        model: ML Model to be checked
        data: data on which the model needs to pe trained
        y: data containing the target variables
        predictors: independent feature variable
        target: target variable
    '''
    model.fit(data[predictors].copy(), y[target].copy())
    predictions = model.predict(data[predictors].copy())
    rms = sklearn.metrics.mean_squared_error(predictions,y[target])**0.5
    #print('RMS : %s' % '{0:.2%}'.format(rms))
    r2 = sklearn.metrics.r2_score(predictions,y[target])
    #print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [139]:
def winner_prediction_scores(model, data, y, predictors,winner):
  '''
    A generic function to predict the winner for the model in question 
    
    Args:
        model: ML Model to be checked
        data: data on which the model needs to be trained
        y: data containing the target variables
        predictors: independent feature variable
        winner: winning team
  '''
  
  pred1 = print_model_scores(model, X_train, y_train,predictor_var, ['Score_1'])
  pred2 = print_model_scores(model, X_train, y_train,predictor_var, ['Score_2'])

  pred = pred1 - pred2

  for i in range(len(pred)):
    if ((pred[i])>0):
      pred[i] = int(1)
    else:
      pred[i] = int(2)

  pred = pred.ravel()
  pred = pred.astype(np.int32)
  winner = np.asarray(winner).astype(np.int32)
  try:
    print("based on Scores: ")
    print(classification_report(winner, pred))
  except:
    print(pred)
    print(pred.dtype)
    print(np.asarray(winner))
    print(np.asarray(winner).dtype)


In [141]:
def winner_prediction_nrr(model, data, y, predictors,winner):
    pred = print_model_scores(model, X_train, y_train,predictor_var, ['NRR_team1'])
    
    
    for i in range(len(pred)):
        if ((pred[i])>0):
            pred[i] = int(1)
        elif (pred[i]==0):
            pred[i] = int(0)
        else:
            pred[i] = int(2)

    pred = pred.astype(np.int32)
    winner = np.asarray(winner).astype(np.int32)

    print("based on NRR: ")
    print(classification_report(winner, pred))

###Model1 - Toss + GroundCode

In [143]:
for x,y,_ in teams_list:

  df = flipped_df.loc[(flipped_df['Team_1'].isin([x])) & (flipped_df['Team_2'].isin([y]))].copy()

  X_data = df[['Date','Team_1Enc','Team_2Enc','Venue','GroundCode','TOSS','Toss1','Toss2']].copy()
  y_data = df[['Winner_team','Win1','Win2','Score_1','Score_2','NRR_team1','NRR_team2']].copy()

  if (len(X_data)==1):
    continue
  X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 1/5)

  print("Classification report for Teams",x,"vs",y)
  winner = y_train['Winner_team'].copy()
  predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
  model = DecisionTreeRegressor()
  winner_prediction_scores(model, X_train, y_train,predictor_var, winner)
  winner_prediction_nrr(model, X_train, y_train,predictor_var, winner)

  print('****************************************************')



Classification report for Teams AFG vs AUS
based on Scores: 
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

based on NRR: 
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

****************************************************
Classification report for Teams AFG vs BAN
based on Scores: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         3

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

based on Scores: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

based on NRR: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

****************************************************
Classification report for Teams AFG vs NZL
based on Scores: 
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


based on NRR: 
              precision    recall  f1-score   support

           1       0.78      0.80      0.79        44
           2       0.61      0.58      0.60        24

    accuracy                           0.72        68
   macro avg       0.69      0.69      0.69        68
weighted avg       0.72      0.72      0.72        68

****************************************************
Classification report for Teams AUS vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.70      0.88      0.78        52
           2       0.83      0.60      0.70        50

    accuracy                           0.75       102
   macro avg       0.77      0.74      0.74       102
weighted avg       0.76      0.75      0.74       102

based on NRR: 
              precision    recall  f1-score   support

           1       0.69      0.88      0.77        52
           2       0.83      0.58      0.68        50

    accuracy                          

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


based on Scores: 
              precision    recall  f1-score   support

           1       0.84      0.75      0.79        55
           2       0.73      0.82      0.77        45

    accuracy                           0.78       100
   macro avg       0.78      0.78      0.78       100
weighted avg       0.79      0.78      0.78       100

based on NRR: 
              precision    recall  f1-score   support

           1       0.80      0.71      0.75        55
           2       0.69      0.78      0.73        45

    accuracy                           0.74       100
   macro avg       0.74      0.74      0.74       100
weighted avg       0.75      0.74      0.74       100

****************************************************
Classification report for Teams WIN vs SAF
based on Scores: 
              precision    recall  f1-score   support

           1       0.88      0.58      0.70        12
           2       0.86      0.97      0.91        32

    accuracy                       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


based on Scores: 
              precision    recall  f1-score   support

           1       0.73      0.50      0.59        22
           2       0.74      0.89      0.81        36

    accuracy                           0.74        58
   macro avg       0.74      0.69      0.70        58
weighted avg       0.74      0.74      0.73        58

based on NRR: 
              precision    recall  f1-score   support

           1       0.73      0.50      0.59        22
           2       0.74      0.89      0.81        36

    accuracy                           0.74        58
   macro avg       0.74      0.69      0.70        58
weighted avg       0.74      0.74      0.73        58

****************************************************
Classification report for Teams ZIM vs ENG
based on Scores: 
              precision    recall  f1-score   support

           1       1.00      0.71      0.83         7
           2       0.87      1.00      0.93        13

    accuracy                       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


###Model2 - Toss + GroundCode + Venue

In [144]:
for x,y,_ in teams_list:

  df = flipped_df.loc[(flipped_df['Team_1'].isin([x])) & (flipped_df['Team_2'].isin([y]))].copy()

  X_data = df[['Date','Team_1Enc','Team_2Enc','Venue','GroundCode','TOSS','Toss1','Toss2']].copy()
  y_data = df[['Winner_team','Win1','Win2','Score_1','Score_2','NRR_team1','NRR_team2']].copy()

  if (len(X_data)==1):
    continue
  X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 1/5)

  print("Classification report for Teams",x,"vs",y)
  winner = y_train['Winner_team'].copy()
  predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
  model = DecisionTreeRegressor()
  winner_prediction_scores(model, X_train, y_train,predictor_var, winner)
  winner_prediction_nrr(model, X_train, y_train,predictor_var, winner)

  print('****************************************************')



Classification report for Teams AFG vs AUS
based on Scores: 
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

based on NRR: 
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

****************************************************
Classification report for Teams AFG vs BAN
based on Scores: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         4

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

based on NRR: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

****************************************************
Classification report for Teams AFG vs NZL
based on Scores: 
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


based on Scores: 
              precision    recall  f1-score   support

           1       0.67      0.89      0.76        54
           2       0.80      0.50      0.62        48

    accuracy                           0.71       102
   macro avg       0.73      0.69      0.69       102
weighted avg       0.73      0.71      0.69       102

based on NRR: 
              precision    recall  f1-score   support

           1       0.67      0.78      0.72        54
           2       0.69      0.56      0.62        48

    accuracy                           0.68       102
   macro avg       0.68      0.67      0.67       102
weighted avg       0.68      0.68      0.67       102

****************************************************
Classification report for Teams AUS vs ZIM
based on Scores: 
              precision    recall  f1-score   support

           1       0.95      1.00      0.98        21
           2       1.00      0.50      0.67         2

    accuracy                       

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


based on Scores: 
              precision    recall  f1-score   support

           1       0.82      0.75      0.79        56
           2       0.71      0.80      0.75        44

    accuracy                           0.77       100
   macro avg       0.77      0.77      0.77       100
weighted avg       0.78      0.77      0.77       100

based on NRR: 
              precision    recall  f1-score   support

           1       0.82      0.71      0.76        56
           2       0.69      0.80      0.74        44

    accuracy                           0.75       100
   macro avg       0.75      0.75      0.75       100
weighted avg       0.76      0.75      0.75       100

****************************************************
Classification report for Teams WIN vs SAF
based on Scores: 
              precision    recall  f1-score   support

           1       0.70      0.64      0.67        11
           2       0.88      0.91      0.90        33

    accuracy                       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


vs BAN
based on Scores: 
              precision    recall  f1-score   support

           1       0.86      0.50      0.63        24
           2       0.73      0.94      0.82        34

    accuracy                           0.76        58
   macro avg       0.79      0.72      0.73        58
weighted avg       0.78      0.76      0.74        58

based on NRR: 
              precision    recall  f1-score   support

           1       0.86      0.50      0.63        24
           2       0.73      0.94      0.82        34

    accuracy                           0.76        58
   macro avg       0.79      0.72      0.73        58
weighted avg       0.78      0.76      0.74        58

****************************************************
Classification report for Teams ZIM vs ENG
based on Scores: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00        15

    accuracy                

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

##Classification Report on Team vs All

In [147]:
team_list = flipped_df.groupby(["Team_1"]).size().reset_index(name="Matches")

In [148]:
team_list.head()

,Team_1,Matches
0,AFG,102
1,AUS,879
2,BAN,346
3,ENG,676
4,IND,885


In [150]:
teams_list = team_list.values.tolist()

In [151]:
print(teams_list)

[['AFG', 102], ['AUS', 879], ['BAN', 346], ['ENG', 676], ['IND', 885], ['IRE', 121], ['KEN', 123], ['NED', 56], ['NZL', 692], ['PAK', 849], ['SAF', 558], ['SCO', 68], ['SRL', 753], ['WIN', 716], ['ZIM', 474]]


In [152]:
def print_model_scores(model, data, y, predictors, target):
    '''
    A generic function to generate the performance report of the
    model in question on the data passed to it
    Args:
        model: ML Model to be checked
        data: data on which the model needs to pe trained
        y: data containing the target variables
        predictors: independent feature variable
        target: target variable
    '''
    model.fit(data[predictors].copy(), y[target].copy())
    predictions = model.predict(data[predictors].copy())
    rms = sklearn.metrics.mean_squared_error(predictions,y[target])**0.5
    #print('RMS : %s' % '{0:.2%}'.format(rms))
    r2 = sklearn.metrics.r2_score(predictions,y[target])
    #print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [153]:
def winner_prediction_scores(model, data, y, predictors,winner):
  '''
    A generic function to predict the winner for the model in question 
    
    Args:
        model: ML Model to be checked
        data: data on which the model needs to be trained
        y: data containing the target variables
        predictors: independent feature variable
        winner: winning team
  '''
  
  pred1 = print_model_scores(model, X_train, y_train,predictor_var, ['Score_1'])
  pred2 = print_model_scores(model, X_train, y_train,predictor_var, ['Score_2'])

  pred = pred1 - pred2

  for i in range(len(pred)):
    if ((pred[i])>0):
      pred[i] = int(1)
    else:
      pred[i] = int(2)

  pred = pred.ravel()
  pred = pred.astype(np.int32)
  winner = np.asarray(winner).astype(np.int32)
  try:
    print("based on Scores: ")
    print(classification_report(winner, pred))
  except:
    print(pred)
    print(pred.dtype)
    print(np.asarray(winner))
    print(np.asarray(winner).dtype)


In [154]:
def winner_prediction_nrr(model, data, y, predictors,winner):
    pred = print_model_scores(model, X_train, y_train,predictor_var, ['NRR_team1'])
    
    
    for i in range(len(pred)):
        if ((pred[i])>0):
            pred[i] = int(1)
        elif (pred[i]==0):
            pred[i] = int(0)
        else:
            pred[i] = int(2)

    pred = pred.astype(np.int32)
    winner = np.asarray(winner).astype(np.int32)

    print("based on NRR: ")
    print(classification_report(winner, pred))

###Model1 - Toss + GroundCode

In [155]:
for x,_ in teams_list:

  df = flipped_df.loc[(flipped_df['Team_1'].isin([x]))].copy()

  X_data = df[['Date','Team_1Enc','Team_2Enc','Venue','GroundCode','TOSS','Toss1','Toss2']].copy()
  y_data = df[['Winner_team','Win1','Win2','Score_1','Score_2','NRR_team1','NRR_team2']].copy()

  if (len(X_data)==1):
    continue
  X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 1/5)

  print("Classification report for Teams",x,"vs",y)
  winner = y_train['Winner_team'].copy()
  predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
  model = DecisionTreeRegressor()
  winner_prediction_scores(model, X_train, y_train,predictor_var, winner)
  winner_prediction_nrr(model, X_train, y_train,predictor_var, winner)

  print('****************************************************')



Classification report for Teams AFG vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.88      0.92      0.90        39
           2       0.93      0.88      0.90        42

    accuracy                           0.90        81
   macro avg       0.90      0.90      0.90        81
weighted avg       0.90      0.90      0.90        81

based on NRR: 
              precision    recall  f1-score   support

           1       0.84      0.92      0.88        39
           2       0.92      0.83      0.88        42

    accuracy                           0.88        81
   macro avg       0.88      0.88      0.88        81
weighted avg       0.88      0.88      0.88        81

****************************************************
Classification report for Teams AUS vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.78      0.82      0.80       437
           2       0.68      0.61      0.64    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


based on Scores: 
              precision    recall  f1-score   support

           1       0.88      0.86      0.87        42
           2       0.89      0.91      0.90        54

    accuracy                           0.89        96
   macro avg       0.88      0.88      0.88        96
weighted avg       0.89      0.89      0.89        96

based on NRR: 
              precision    recall  f1-score   support

           1       0.90      0.88      0.89        42
           2       0.91      0.93      0.92        54

    accuracy                           0.91        96
   macro avg       0.91      0.90      0.90        96
weighted avg       0.91      0.91      0.91        96

****************************************************
Classification report for Teams KEN vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.76      0.86      0.81        22
           2       0.96      0.92      0.94        76

    accuracy                       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model2 - Toss + GroundCode + Venue

In [158]:
for x,_ in teams_list:

  df = flipped_df.loc[(flipped_df['Team_1'].isin([x]))].copy()

  X_data = df[['Date','Team_1Enc','Team_2Enc','Venue','GroundCode','TOSS','Toss1','Toss2']].copy()
  y_data = df[['Winner_team','Win1','Win2','Score_1','Score_2','NRR_team1','NRR_team2']].copy()

  if (len(X_data)==1):
    continue
  X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 1/5)

  print("Classification report for Teams",x,"vs",y)
  winner = y_train['Winner_team'].copy()
  predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
  model = DecisionTreeRegressor()
  winner_prediction_scores(model, X_train, y_train,predictor_var, winner)
  winner_prediction_nrr(model, X_train, y_train,predictor_var, winner)

  print('****************************************************')



Classification report for Teams AFG vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.77      0.83      0.80        36
           2       0.86      0.80      0.83        45

    accuracy                           0.81        81
   macro avg       0.81      0.82      0.81        81
weighted avg       0.82      0.81      0.82        81

based on NRR: 
              precision    recall  f1-score   support

           1       0.75      0.83      0.79        36
           2       0.85      0.78      0.81        45

    accuracy                           0.80        81
   macro avg       0.80      0.81      0.80        81
weighted avg       0.81      0.80      0.80        81

****************************************************
Classification report for Teams AUS vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.77      0.86      0.81       434
           2       0.72      0.59      0.65    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


based on Scores: 
              precision    recall  f1-score   support

           1       0.85      0.87      0.86        39
           2       0.91      0.89      0.90        57

    accuracy                           0.89        96
   macro avg       0.88      0.88      0.88        96
weighted avg       0.89      0.89      0.89        96

based on NRR: 
              precision    recall  f1-score   support

           1       0.85      0.87      0.86        39
           2       0.91      0.89      0.90        57

    accuracy                           0.89        96
   macro avg       0.88      0.88      0.88        96
weighted avg       0.89      0.89      0.89        96

****************************************************
Classification report for Teams KEN vs WIN
based on Scores: 
              precision    recall  f1-score   support

           1       0.83      0.86      0.84        22
           2       0.96      0.95      0.95        76

    accuracy                       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
